In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-collaborative-filtering-recommender/small_movies_X.csv
/kaggle/input/movie-collaborative-filtering-recommender/small_movies_b.csv
/kaggle/input/movie-collaborative-filtering-recommender/small_movies_Y.csv
/kaggle/input/movie-collaborative-filtering-recommender/small_movie_list.csv
/kaggle/input/movie-collaborative-filtering-recommender/small_movies_W.csv
/kaggle/input/movie-collaborative-filtering-recommender/small_movies_R.csv


In [2]:
from numpy import loadtxt
import tensorflow as tf
from tensorflow import keras

In [3]:
df = pd.read_csv('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_X.csv')

In [4]:
df.head()

,-0.033286825,1.1667464,-0.5064895,0.1250965,1.5347596,-0.21013564,0.10417578,-0.349303,0.75385165,-0.20810875
0,0.404058,2.266492,-0.742824,1.374039,1.145788,0.381959,-0.447495,-1.038103,-2.332266,-2.343191
1,0.023478,0.428102,-0.426381,0.107177,0.484193,-0.969298,-0.748563,-0.293784,0.439730,1.823422
2,0.511281,1.518797,1.602747,-2.296399,-0.570383,-0.480868,-0.662502,1.965215,0.495176,1.044845
3,-2.056514,0.458517,0.619371,-0.138456,0.011926,-0.282415,0.027502,-0.535465,0.661082,0.195084
4,-0.443277,-0.015877,0.239043,-1.300772,0.100523,-1.271912,0.289156,2.173626,0.203151,-0.522043


In [5]:
def load_params():
    file = open('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_X.csv', 'rb')
    X=loadtxt(file, delimiter = ",")

    file = open('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_W.csv', 'rb')
    W=loadtxt(file, delimiter = ",")

    file = open('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_b.csv', 'rb')
    b=loadtxt(file, delimiter = ",")
    b.reshape(1,-1)

    nm, n = X.shape
    nu = W.shape[0]

    return (X, W, b, nm, n, nu)

In [6]:

def load_ratings():
    file = open('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_Y.csv', 'rb')
    Y=loadtxt(file, delimiter = ",")

    file = open('/kaggle/input/movie-collaborative-filtering-recommender/small_movies_R.csv', 'rb')
    R=loadtxt(file, delimiter = ",")

    return (Y,R) 

In [7]:
X, W, b, nm, n, nu = load_params()
Y, R = load_ratings()

In [8]:
print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", n)
print("num_movies",   nm)
print("num_users",    nu)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (443,)
num_features 10
num_movies 4778
num_users 443


In [9]:
def cost_fun(X, W, b, Y, R, lambda_):
    nm, nu = Y.shape
    n = X.shape[1]
    J = 0
    ### START CODE HERE ###  
    for j in range(nu):
        for i in range(nm):
            J+=R[i][j]*(np.square(np.dot(W[j], X[i]) + b[0][j] - Y[i][j]))
    
    J/=2
    
    reg=0
    
    reg += (np.sum(np.square(W)) + np.sum(np.square(X)))
    reg *= (lambda_/2)
          
    J+=reg
    return J
    

In [10]:
def load_movies():
    df = pd.read_csv("/kaggle/input/movie-collaborative-filtering-recommender/small_movie_list.csv", header=0, index_col=0, delimiter=",", quotechar='"')
    mlist = df["title"].to_list()
    return (mlist, df)

In [11]:
def cost_fun(X, W, b, Y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * tf.reduce_sum(X**2) + (lambda_/2) * tf.reduce_sum(W**2) 
    return J

In [12]:
movies, movies_df = load_movies()

In [13]:
movies[:5]
movies_df.head()

,mean rating,number of ratings,title
0,3.400000,5,"Yards, The (2000)"
1,3.250000,6,Next Friday (2000)
2,2.000000,4,Supernova (2000)
3,2.000000,4,Down to You (2000)
4,2.672414,29,Scream 3 (2000)


In [14]:
my_ratings = np.zeros(nm)

my_ratings[51] = 5
my_ratings[62] = 4
my_ratings[929] = 3.5
my_ratings[2716] = 4.5
my_ratings[366] = 5
my_ratings[2925] = 1
my_ratings[793] = 4.5
my_ratings[382] = 2

rated = [i for i in range(nm) if my_ratings[i]>0]

print("New User Ratings: \n")
for i in range(nm):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movies[i]}');


New User Ratings: 

Rated 5.0 for  Gladiator (2000)
Rated 4.0 for  Mission: Impossible II (2000)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 4.5 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 3.5 for  Lord of the Rings: The Return of the King, The (2003)
Rated 4.5 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)


In [15]:
Y.shape


(4778, 443)

In [16]:
Y = np.c_[my_ratings, Y]
Y.shape

(4778, 444)

In [17]:
R = np.c_[(my_ratings!=0).astype(int), R]
R.shape

(4778, 444)

In [18]:
def normalize_ratings(Y, R):
    Y_mean =(np.sum(Y * R, axis=1)/(np.sum(R, axis=1) +1e-12)).reshape(-1,1)
    Y_norm = Y - np.multiply(Y_mean, R)
    return Y_norm, Y_mean

In [19]:
Y_norm, Y_mean = normalize_ratings(Y, R)
Y_norm.shape

(4778, 444)

In [20]:
W[:5]

array([[ 0.47935075,  0.4766997 ,  0.48794055, -0.11980432, -0.32864022,
         0.4232462 , -0.3951074 , -0.2383076 , -0.36121953, -0.18090296],
       [ 0.03604829, -0.38190877,  0.2958575 , -0.25823373,  0.28605568,
        -0.1814661 ,  0.29349202,  0.4640792 ,  0.45832908, -0.23635024],
       [ 0.05561632, -0.05899394, -0.2452817 ,  0.10987079,  0.3970989 ,
         0.3636213 ,  0.14597559,  0.36375767,  0.01846844,  0.17488128],
       [ 0.4550811 ,  0.15987432, -0.4527412 ,  0.23575765,  0.2386728 ,
        -0.27724636,  0.3670869 , -0.32793385,  0.28871763,  0.37041497],
       [-0.0171572 , -0.43986136, -0.33766657,  0.18368888, -0.45843637,
         0.171238  , -0.36026502,  0.11101794,  0.04873836, -0.43986273]])

In [21]:
tf.random.set_seed(1234)
n=100

W = tf.Variable(tf.random.normal((nu+1, n), dtype=tf.float64), name='W')
X = tf.Variable(tf.random.normal((nm, n), dtype=tf.float64), name='X')
b = tf.Variable(tf.random.normal((1, nu+1), dtype=tf.float64), name='b')

optimizer = keras.optimizers.Adam(learning_rate = 1e-1)

In [22]:
iters = 200
lambda_ = 1
for i in range(iters):
    with tf.GradientTape() as tape:
        cost_value = cost_fun(X, W, b, Y_norm, R, lambda_)

    grads = tape.gradient(cost_value, [X,W,b])
    optimizer.apply_gradients(zip(grads, [X,W,b]))

    if i%20 == 0:
        print(f"Training loss at iteration {i} : {cost_value:0.1f}")

Training loss at iteration 0 : 2321113.2
Training loss at iteration 20 : 136156.0
Training loss at iteration 40 : 51854.0
Training loss at iteration 60 : 24593.7
Training loss at iteration 80 : 13627.4
Training loss at iteration 100 : 8485.8
Training loss at iteration 120 : 5806.5
Training loss at iteration 140 : 4310.8
Training loss at iteration 160 : 3434.6
Training loss at iteration 180 : 2901.5


In [23]:
p = np.matmul(X, np.transpose(W)) + b
pm = p + Y_mean
pm_clipped = np.clip(pm, 0, 5)
my_preds = pm_clipped[:,0]

ix = tf.argsort(my_preds, direction='DESCENDING')

for i in range(30):
    j = ix[i]
    if j not in rated:
        print(f'Predicted {my_preds[j]:0.2f} for {movies[j]}')

print('\n\nOriginal vs Predicted Ratings:')
for i in range(nm):
    if my_ratings[i]>0:
        print(f'Original {my_ratings[i]}, Predicted {my_preds[i]:0.2f} for {movies[i]}')

Predicted 4.45 for My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicted 4.43 for Odd Life of Timothy Green, The (2012)
Predicted 4.43 for Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicted 4.43 for Eichmann (2007)
Predicted 4.43 for Into the Abyss (2011)
Predicted 4.42 for Particle Fever (2013)
Predicted 4.41 for 61* (2001)
Predicted 4.41 for Wonder Woman (2009)
Predicted 4.41 for Rivers and Tides (2001)
Predicted 4.41 for Eva (2011)
Predicted 4.41 for Paper Birds (Pájaros de papel) (2010)
Predicted 4.41 for Strictly Sexual (2008)
Predicted 4.41 for Battle For Sevastopol (2015)
Predicted 4.41 for Bitter Lake (2015)
Predicted 4.40 for The Girl with All the Gifts (2016)
Predicted 4.40 for SORI: Voice from the Heart (2016)
Predicted 4.40 for Che: Part Two (2008)
Predicted 4.40 for Che: Part One (2008)
Predicted 4.40 for Peaceful Warrior (2006)
Predicted 4.40 for Tom Segura: Completely Normal (2014)
Predicted 4.40 for Tom Segura: Mostly Stories (2016)
Predicted 4.40

In [24]:
filter = movies_df["number of ratings"]>20
movies_df["pred"] = my_preds
movies_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,mean rating,number of ratings,title,pred
2112,4.238255,149,"Dark Knight, The (2008)",4.046680
211,4.122642,159,Memento (2000),4.237861
1318,4.075000,40,Howl's Moving Castle (Hauru no ugoku shiro) (2...,4.050690
2716,4.066434,143,Inception (2010),4.445232
51,3.938235,170,Gladiator (2000),4.901510
2455,3.887931,58,Harry Potter and the Half-Blood Prince (2009),3.986637
3014,3.869565,69,"Avengers, The (2012)",4.024506
793,3.778523,149,Pirates of the Caribbean: The Curse of the Bla...,4.408992
366,3.761682,107,Harry Potter and the Sorcerer's Stone (a.k.a. ...,4.796919
79,3.699248,133,X-Men (2000),4.088463


In [25]:
np.unique(pm_clipped, return_counts=True)

(array([0.00000000e+00, 4.49390900e-05, 7.33990358e-05, ...,
        4.99999728e+00, 4.99999947e+00, 5.00000000e+00]),
 array([ 3665,     1,     1, ...,     1,     1, 72313]))